## Task 1:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#from statsmodels.stats.outliers_influence import variance_inflation_factor

_ add to back later _
For the final analysis, we chose to use policy responses to group the type of characteristic groups. 
**stringency_index**

**Read the CSV file** owid_covid_data.csv
extracted from: https://github.com/owid/covid-19-data/tree/master/public/data/

As expected of the question, **confirmed deaths** was used as the target metric.
Target = df["total_deaths_per_million"]

To determine the right set of variables for the learning phase, we had to look for independent variables that were had reduced multicollinearity data since that would not increase prediction accuracy. 

For the first step of cutting, we simply took all cumulative data the dataset offered for each metric used. For the second round of elimination, we chose to use a correlation heatmap offered by heatmap to provide us with a visual depiction of the relationship between variables.

**First feature set**:
- `Vaccinations: total_vaccinations and people_fully_vaccinated`
- `Tests & positivity: positive_rate`
- `Hospital & ICU: icu_patients and hosp_patients`
- `Confirmed cases: total_cases`
- `Excess Mortality: excess_mortality_cumulative_absolute`

(maybe group countries based on whether they are "first world, second world or third world" as determined by policies?)


In [ ]:
df = pd.read_csv("owid_covid_data.csv")
display(df)

In [ ]:
df_first_features = df[["total_vaccinations", "people_fully_vaccinated", "positive_rate", "icu_patients", "hosp_patients", "total_cases", "excess_mortality_cumulative_absolute"]]

sns.heatmap(df_first_features.corr(), annot=True, cmap="YlGnBu")
plt.show()

In [ ]:
#maybe can delete first
#plot the number of deaths to determine an ideal time period
df_deaths = df[["total_deaths", "date"]]
df_deaths = df_deaths.drop_duplicates(subset=['date'])

plt.scatter(df_deaths.index, df_deaths["total_deaths"])

# find POI = 0 (second derivative test)
display(df_deaths)
df_deaths[["first_derivative"]] = df_deaths["total_deaths"]
df_deaths[["first_derivative"]] = df_deaths["first_derivative"].shift()
df_deaths[["first_derivative"]] = df_deaths["first_derivative"]-df_deaths["total_deaths"]

df_deaths[["second_derivative"]] = df_deaths["first_derivative"]
df_deaths[["second_derivative"]] = df_deaths["second_derivative"].shift()
df_deaths[["second_derivative"]] = df_deaths["second_derivative"]-df_deaths["first_derivative"]
display(df_deaths)
plt.scatter(df_deaths.index, df_deaths["first_derivative"])
plt.scatter(df_deaths.index, df_deaths["second_derivative"])

In [ ]:
#necessary data cleaning:
df["time"] = pd.to_datetime(df['dates'], format = %Y%m%d)

### Hypothesis 1:

A initial hypothesis we make assumes that quantity of policies undertaken by a country crucially affects the number of deaths in that country. As such, we group the countries into 4 large subsets. 

In [ ]:
df_policies = df[["stringency_index", "location"]]
df_policies = df_policies.drop_duplicates(subset=['location'])

#plt.scatter(df_policies["stringency_index"], df_policies["location"])
display(df_policies.describe())
#plt.scatter(df_policies.index, df_policies["stringency_index"])

divider1, divider2, divider3 = df_policies.describe().loc["25%", "50%", "75%"]
print(divider1, divider2, divider3)

country_subset1 = df_policies.loc[(df_policies["stringency_index"] < divider1), ["location"]]
print(display(country_subset1))

In [ ]:
df_second_features = df[["people_fully_vaccinated", "positive_rate", "hosp_patients", "total_cases", "excess_mortality_cumulative_absolute"]]

sns.heatmap(df_second_features.corr(), annot=True, cmap="YlGnBu")
plt.show()

### Before Cleaning the data, here are some functions to be used
**Functions to be used** These are the functions that will be used
- `normalize_z()`
- `get_features_targets()`
- `prepare_feature()`
- `prepare_target()`
- `predict()`
- `predict_norm()`
- `split_data()`
- `mean_squared_error()`

In [ ]:
def normalize_z(df):
    dfout = (df-df.mean(axis=0))/(df.std(axis=0))
    return dfout

def get_features_targets(df, feature_names, target_names):
    df_feature = df[feature_names]
    df_target = df[target_names]
    return df_feature, df_target

def prepare_feature(df_feature):
    df1 = df_feature
    array1 = df_feature.to_numpy()
    array2 = np.ones((array1.shape[0], 1))
    return np.concatenate((array2, array1), axis=1) #axis 1 means side by side (column)
    
def prepare_target(df_target):
    array1 = df_target.to_numpy()
    return array1

def predict(df_feature, beta):
    df_feature = normalize_z(df_feature)
    df_feature = prepare_feature(df_feature)
    result = predict_norm(df_feature, beta)
    return result

def predict_norm(X, beta):
    # yhat = X x beta
    array = np.matmul(X, beta)
    return array

def split_data(df_feature, df_target, random_state=100, test_size=0.3):
    np.random.seed(random_state)
    
    values = np.random.choice(df_feature.index,int(len(df_feature)*test_size), replace = False)
    anti_values = [i for i in df_feature.index if i not in values]

    df_feature_test = df_feature.loc[values,:]
    df_feature_train = df_feature.loc[anti_values, :]
    
    df_target_test = df_target.loc[values,:]
    df_target_train = df_target.loc[anti_values,:]
    
    return df_feature_train, df_feature_test, df_target_train, df_target_test

def mean_squared_error(target, pred):
    error = target-pred
    
    return (1/(target.shape[0]))* np.matmul(error.T, error)
    

Based on the second heatmap, will be extracting the following columns:

**Final feature set**: takes one from each matric
- `Vaccinations: people_fully_vaccinated`
- `Tests & positivity: positive_rate`
- `Hospital & ICU: hosp_patients`
- `Confirmed cases: total_cases`
- `Excess Mortality: excess_mortality_cumulative_absolute`

features = "people_fully_vaccinated", "positive_rate", "hosp_patients", "total_cases", "excess_mortality_cumulative_absolute" 

target = "total_deaths_per_million"

Normalize the features using z normalization.
Plot the data using scatter plot. Use the following columns:

In [ ]:
# Extract the features and the target
df_features, df_target = get_features_targets(df, ["people_fully_vaccinated", "positive_rate", "hosp_patients", "total_cases", "excess_mortality_cumulative_absolute"], ["total_deaths_per_million"])

#clean the data: replace NaN with 0
df_features = df_features.fillna(0)

# Split the data set into training and test
df_features_train, df_features_test, df_target_train, df_target_test = (split_data(df_features, df_target, random_state=100, test_size=0.3))

# Normalize the features using z normalization
df_features_train_z = normalize_z(df_features_train)

# Change the features and the target to numpy array using the prepare functions
X = prepare_feature(df_features_train_z)
target = prepare_target(df_target_train)

In [ ]:
display(df_features_train)

In [ ]:
display(df_features_train.describe())

In [ ]:
def compute_cost(X, y, beta):
    J = 0
    error = np.matmul(X, beta) - y
    #print((1/(2*X.shape[0])))
    J = (1/(2*X.shape[0]))*(np.matmul(error.T, error))
    return J

def gradient_descent(X, y, beta, alpha, num_iters):
    m = X.shape[0]
    for i in range(num_iters):
        yhat = np.matmul(X, beta)
        beta = beta - ((alpha/m)*np.matmul(X.T, (yhat-y)))
    
    print(beta)
    return beta

In [ ]:
iterations = 1500
alpha = 0.01
beta = np.zeros((6,1))

# Call the gradient_descent function
beta = gradient_descent(X, target, beta, alpha, iterations)
print("2", beta)

# call the predict() method
pred = predict(df_features_test, beta)

###
print(pred.shape)
print(pred.mean())
print(pred.std())
###

In [ ]:
plt.scatter(df_feature, df_target)

### Iteration 3: (bryan's addition)

Since we were doing a future prediction model, we decided to predict for day n+1 instead of for day n. Total number of deaths/ Targets of intervention is hence displaced by 1.